# PySparkを実際に本番環境で動かす際の流れ
ローカル端末で動かすことはわかったので、本チャプターはPySparkを実際に本番環境で実行するためのTipsを紹介していきます。  
メインのトピックとしては、

- スケジューラーで実行するためにコマンドベースでどのように動かすか？
- チューニングする際に利用するツールやそのコツ

を紹介していきます。

# 本チャプターの目次

2. SQLファイルの読み込みと実行
3. Spark Submit(Sparkをコマンドラインで実行する方法)
1. Spark Web インタフェース
5. チューニングのコツ

# spark-submit でSparkを実行する

コマンドラインでSparkを実行するにはこの方法しかない。
構文としては

```
spark-submit xxxxx.py 引数 
```

で実行可能

# Sparkを用いたバッチにおけるデータエンジニアリング一連の流れ(復習)

1. データソースの読み込み(今回は、人口統計データ(/dataset/jinkou.csv))　ETL(Extract Transform Load)で言うEの部分
2. 変換を行う(集計等を行う)　DataFrame処理 or SQL処理の２パターンで実行可能 ETL(Extract Transform Load)で言うTの部分
3. カラムナーフォーマットへ変換する ETL(Extract Transform Load)で言うTの部分
4. 出力したデータをみんなに見やすくするため(BIツールから参照できるように)テーブルを作成する ETL(Extract Transform Load)で言うLの部分

本チャプターではこの流れを、Spark-Submitを使った方法で実行してみようと思います。
ただし、テーブルは事前に作成しておくことにします。

In [1]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は次のチャプターで説明を行います。
spark = SparkSession.builder \
    .appName("chapter3") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()

21/09/09 15:36:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
# テーブルを作成しておきます
spark.sql(""" 
CREATE EXTERNAL TABLE IF NOT EXISTS default.jinko_avg ( male_avg double, female_avg double)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
LOCATION '/Users/yuki/pyspark_batch/dataset/parquet/';
""")
spark.sql(" show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|jinko_avg|      false|
+--------+---------+-----------+



In [ ]:
# spark_etl.pyを作成していきます
# ここからはノートブックではなくpythonエディターへ移ります。

In [7]:
# Spark利用の停止
spark.stop()
spark.sparkContext.stop()